# Studio Embedding

In [ ]:
from model_analysis import plot_embeddings_umap

plot_embeddings_umap(
    'output/val_embeddings/val_all_merged.npz',
    color_by='labels',
    output_path='output/val_embeddings/val_umap_labels.png',
    show=False
)


Impossibile caricare la libreria torch dynamo


In [ ]:
%matplotlib widget

import numpy as np
import umap
import matplotlib.pyplot as plt

d = np.load('output/val_embeddings/val_all_merged.npz', allow_pickle=True)
X = d['embeddings']
labels = d['labels'] if 'labels' in d else None
preds  = d['preds']  if 'preds'  in d else None

# Choose coloring
color = labels if labels is not None else (preds if preds is not None else None)

reducer = umap.UMAP(n_components=3, random_state=42, n_neighbors=15, min_dist=0.1, metric='euclidean')
Z = reducer.fit_transform(X)  # shape (N, 3)

N, D = X.shape
title = f"UMAP embeddings 3D (N={N}, dim={D})"

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(Z[:, 0], Z[:, 1], Z[:, 2],
                c=color if color is not None else 'tab:blue',
                s=4, cmap='Spectral', alpha=0.8)
if color is not None:
    cbar = fig.colorbar(sc, ax=ax, pad=0.1)
    cbar.set_label('labels' if labels is not None else 'preds')

ax.set_title(title)
ax.set_xlabel('UMAP-1'); ax.set_ylabel('UMAP-2'); ax.set_zlabel('UMAP-3')
plt.tight_layout()
plt.savefig('output/val_embeddings/val_umap3d_labels.png', dpi=200)

title


ModuleNotFoundError: No module named 'ipympl'